### Libraries

In [1]:
import sys
sys.path.insert(0,'/Users/jarad/Fake Folder/Python Libraries')

from jb_libraries import *
from part_cost_data import *
%matplotlib inline


== part_cost_data start ==

0 nulls
the mean_stripe_margin is 76.63%

the part types with no MSRP are/is: ['sku']

avg gross profit for:
parts, combos, and stripes
which have been purchased within the last year
whose sku_status equals "working"
whose msrp is greater than zero
which were bought by non resellers
is 55.60%

the parts with negative gross profit are: [2885, 3400, 3834]

your dfs are: all_cost (cost on the sku level) and all_cost_by_part (cost on the part level)

== part_cost_data end ==



### Script settings

In [2]:
# go two years back for main sales data, then restrict to only one year in your "data" df
date_start = '2017-04-01'
date_end = '2019-04-30'

close_workbook = 'yes'
close_plot = 'yes'

resellers = {'Chicago Electronic Distributors':170955,
             'Elmwood Electronics':402263,
             'Digi-Key':515404,
             'Arrow':430980,
             'Mouser':383922,
             'Akizuki Denshi Tsusho Co.':297368,
             'Exp Tech':128228,
             'Pi Hut':218531,
             'KJD Electronics':211704,
             'Pimoroni':122936,
             'Distrelec':654035,
             'Micro Center':390641,
             'Allied Electronics':489596,
             'Eckstein':430625}

one_year_ago = pd.to_datetime(date_start) + pd.DateOffset(years = 1)

year_and_month = pd.DataFrame({'year and month':pd.to_datetime(pd.date_range(one_year_ago, date_end, freq = 'MS'))})
year_and_month['year and month'] = [str(x)[:7] for x in year_and_month['year and month']]
    
rolling_quarter_start = str(pd.to_datetime(date_end) - pd.DateOffset(days = 90))[:7] + '-01'
rolling_quarter_end = date_end

pretty_rolling_quarter_start = jb_dates(pd.Series(rolling_quarter_start), 'year and month pretty')[0]
pretty_rolling_quarter_end = jb_dates(pd.Series(rolling_quarter_end), 'year and month pretty')[0]

csv_path = '/Users/jarad/Fake Folder/Accounts and Biz Dev/Recurring/Reseller Report/CSVs/'
workbook_title = date_end[:4] + ' - ' + date_end[5:7] + ' - ' + calendar.month_abbr[int(date_end[5:7])] + ' - Reseller Report'

### View orders_statuses

In [3]:
pd.read_sql(
'''
SELECT
*
FROM orders_status
ORDER BY orders_status_id
''', db)

,orders_status_id,language_id,orders_status_name
0,1,1,Pending
1,2,1,Processing
2,3,1,Shipped
3,4,1,Update
4,5,1,Printed
5,6,1,Billed
6,7,1,Payment Received
7,8,1,Fraud - Pending
8,9,1,Fraud - Confirmed
9,10,1,Return


### Get excluded parts

In [4]:
discontinued = pd.read_sql(
'''
SELECT
p.part_id
FROM parts p
LEFT JOIN products_description pd ON p.part_id = pd.part_id
WHERE p.products_discontinued = 1 OR p.products_dont_sell_to_resellers = 1
''', db)

internal = pd.read_sql(
'''
SELECT
part_id
FROM skus
WHERE sku_is_internal = 1
AND part_id != 0
''', db)

by_hand = [2843,3358,3359,3360,3067,2931,2929,2684,2893,2481,2877,2904,2905,2907,2908,3075,3126,3074,30,31,32,141,3331,1205,1245,1408,3792]

always_exclude = discontinued['part_id'].tolist() + internal['part_id'].tolist() + by_hand

### Get orders_products data

In [5]:
orders_products = pd.read_sql(
'''
SELECT
DATE(o.date_purchased) AS date,
DATE_FORMAT(o.date_purchased, '%Y-%m') AS year_and_month,
o.orders_id,
o.customers_id,
IF(o.orders_reseller = 0 AND o.orders_super_reseller = 0, 'non reseller', 'reseller') AS customer,
op.part_id,
op.products_quantity - op.products_quantity_free AS qty_bought,
op.products_quantity_free AS qty_free,
op.products_quantity AS qty_total,
(op.products_quantity - op.products_quantity_free) * op.products_price AS revenue,
op.products_price AS price_paid_per_unit

FROM orders_products op
JOIN orders o ON op.orders_id = o.orders_id

WHERE DATE(o.date_purchased) BETWEEN ' '''+ date_start +''' ' AND ' '''+ date_end +''' '
AND o.orders_status NOT IN (8,9,10,11,12,13,14,15)
AND o.payment_method != 'Replacement Order'
''', db)

col_fix(orders_products)

### Make a copy

In [6]:
data = orders_products.copy()

### Map all_cost_by_part data

In [7]:
data['cost per unit'] = data['part id'].map(dict(zip(all_cost_by_part['part id'], all_cost_by_part['latest cost'])))
data['type'] = data['part id'].map(dict(zip(all_cost_by_part['part id'], all_cost_by_part['type'])))

In [8]:
n = data[data.isnull().any(1)]
s = set(n['part id'])

print('{:,.0f} parts with null cost\nthey are: {}\nfill with avg gross profit margin of {:,.2f}%'.format(len(s), s, avg_gp_m * 100))

data['cost per unit'].fillna(data['price paid per unit'] * (1 - avg_gp_m), inplace = True)
data['type'].fillna('virtual/partnership/other', inplace = True)

5 parts with null cost
they are: {3792, 4057, 2843, 4220, 1245}
fill with avg gross profit margin of 55.60%


### Get BOM data

In [9]:
bom = pd.read_sql(
'''
SELECT
part_id,
sku_id,
sku_date_modified,
bom_type AS bom
FROM skus
WHERE part_id != 0
''', db)

col_fix(bom)

bom.sort_values(['part id','sku date modified'], inplace = True)
bom.drop_duplicates('part id', keep = 'last', inplace = True)

In [10]:
data['bom'] = data['part id'].map(dict(zip(bom['part id'], bom['bom'])))

In [11]:
n = data[data.isnull().any(1)]

s = list(set(n[n['bom'].isnull()]['type']))
if (len(s) == 1 or len(s) == 2) and any(x in s for x in ['combo','virtual/partnership/other']):
    data['bom'].fillna(data['type'], inplace = True)
else: 
    raise 1

### Get combined_sales_90 data

In [12]:
combined_sales_90 = pd.read_sql(
'''
SELECT
part_id,
combined_sales_90
FROM parts
''', db)

col_fix(combined_sales_90)

In [13]:
data['combined sales 90'] = data['part id'].map(dict(zip(combined_sales_90['part id'], combined_sales_90['combined sales 90'])))

In [14]:
n = data[data.isnull().any(1)]

if n.empty:
    pass
else:
    raise 1

### Get manufacturing data

In [15]:
manufacturing = pd.read_sql(
'''
SELECT
k.part_id
FROM processes p
JOIN work_orders w ON p.wo_id = w.wo_id
AND wo_outsourced != 1
LEFT JOIN skus k ON w.sku_id = k.sku_id
WHERE process_group = 'fabricate'
''', db)

col_fix(manufacturing)
manufacturing.drop_duplicates('part id', inplace = True)
manufacturing.dropna(inplace = True)
manufacturing['part id'] = [int(x) for x in manufacturing['part id']]

In [16]:
data['manufacturer'] = np.where(data['part id'].isin(manufacturing['part id'].tolist()), 'Adafruit Industries', 'someone else')

### Get discounts data

In [17]:
reseller_discounts = pd.read_sql(
'''
SELECT
part_id,
discount_qty AS reseller_discount_qty,
discount_price / 100 AS 'reseller_max_%_off'
FROM products_reseller_discount_pricing
''', db)

col_fix(reseller_discounts)
reseller_discounts.sort_values(['part id','reseller max % off'], ascending = [True, False], inplace = True)
reseller_discounts.drop_duplicates('part id', keep = 'first', inplace = True)

qty_discounts = pd.read_sql(
'''
SELECT
part_id,
discount_qty AS qty_discount_qty,
discount_price / 100 AS 'qty_max_%_off'
FROM products_discount_quantity
''', db)

col_fix(qty_discounts)
qty_discounts.sort_values(['part id','qty max % off'], ascending = [True, False], inplace = True)
qty_discounts.drop_duplicates('part id', keep = 'first', inplace = True)

In [18]:
for df in [reseller_discounts, qty_discounts]:
    cols = df.columns.tolist()
    cols.remove('part id')
    for c in cols:
        data[c] = data['part id'].map(dict(zip(df['part id'], df[c])))
        data[c].fillna(0, inplace = True)
        if '%' not in c:
            data[c] = [int(x) for x in data[c]]

### Get products_description data

In [19]:
products_description = pd.read_sql(
'''
SELECT
part_id,
products_name
FROM products_description
''', db)

col_fix(products_description)

In [20]:
data['products name'] = data['part id'].map(dict(zip(products_description['part id'], products_description['products name'])))

In [21]:
n = data[data.isnull().any(1)]

if n.empty:
    pass
else:
    raise 1

### Fix datatypes and add some columns

In [22]:
data['date'] = pd.to_datetime(data['date'])

for col in ['qty bought','qty free','qty total']:
    data[col] = [int(x) for x in data[col]]

link_path = 'https://www.adafruit.com/product/'
data['link'] = link_path + data['part id'].map(str)

inv_map = {v: k for k, v in resellers.items()}
data['company'] = data['customers id'].map(inv_map)
data['company'].fillna('the rest', inplace = True)

data['gross profit'] = data['revenue'] - data['cost per unit'] * data['qty total']
data['msrp'] = data['part id'].map(dict(zip(msrp['part id'], msrp['msrp'])))
data['price per unit with reseller max discount'] = data['msrp'] * (1 - data['reseller max % off'])
data['margin with reseller max discount'] = 1 - (data['cost per unit']/data['price per unit with reseller max discount'])
data['price per unit with qty max discount'] = data['msrp'] * (1 - data['qty max % off'])
data['margin with qty max discount'] = 1 - (data['cost per unit']/data['price per unit with qty max discount'])
data['adafruit manufactured'] = np.where(data['manufacturer'] == 'Adafruit Industries', 'x', '')

df = data[data['date'] >= rolling_quarter_start].groupby(['part id','customer'])[['qty bought']].sum().unstack(1)
df.columns = df.columns.droplevel(0)
df['reseller'] = np.where(df['reseller'].isnull() == False, 'x','')
df['all customers'] = np.where((df['reseller'].isnull() == False)
                             & (df['non reseller'].isnull() == False), 'x','')
df.drop('non reseller', 1, inplace = True)

for col in df.columns:
    data[col] = data['part id'].map(dict(zip(df.index.to_series(), df[col])))

### Prepare for Excel

In [23]:
ada_rolling_q_stats = data.copy()
ada_rolling_q_stats = ada_rolling_q_stats[ada_rolling_q_stats['date'].between(rolling_quarter_start, rolling_quarter_end)]

In [24]:
all_cust = ada_rolling_q_stats.groupby(['part id','products name'], as_index = False)[['qty bought']].sum().sort_values('qty bought', ascending = False).head(25)
res_only = ada_rolling_q_stats[ada_rolling_q_stats['customer'] == 'reseller'].groupby(['part id','products name'], as_index = False)[['qty bought']].sum().sort_values('qty bought', ascending = False).head(25)
ada_manufact_only = ada_rolling_q_stats[ada_rolling_q_stats['manufacturer'] == 'Adafruit Industries'].groupby(['part id','products name'], as_index = False)[['qty bought']].sum().sort_values('qty bought', ascending = False).head(25)

In [25]:
data_by_company = data.groupby(['year and month','company'])[['revenue']].sum().unstack(1).fillna(0)
data_by_company.columns = data_by_company.columns.droplevel(0)
data_by_company['row total'] = data_by_company.sum(1)

data_by_company.reset_index(inplace = True)

fmt = [0] + ['m0'] * (len(data_by_company.columns) - 1)
data_by_company.format_(fmt).tail(13)

,Year And Month,Akizuki Denshi Tsusho Co.,Allied Electronics,Arrow,Chicago Electronic Distributors,Digi-Key,Distrelec,Eckstein,Elmwood Electronics,Exp Tech,Kjd Electronics,Micro Center,Mouser,Pi Hut,Pimoroni,The Rest,Row Total
12,2018-04,$0,"$31,975","$31,058","$58,682","$318,936",$0,"$8,906","$5,963","$33,032","$49,181","$97,251","$302,930","$34,031","$24,038","$2,626,265","$3,622,248"
13,2018-05,$0,"$31,429","$45,916","$65,880","$268,726","$78,077","$10,205","$7,080","$27,943","$81,641","$10,537","$205,481","$42,979","$10,444","$2,969,029","$3,855,367"
14,2018-06,$0,"$40,720","$20,764","$60,533","$306,592",$0,"$12,375","$4,677","$43,105","$39,892","$151,435","$154,118","$25,348","$25,686","$2,379,057","$3,264,303"
15,2018-07,$0,"$33,837","$8,486","$46,981","$317,436","$20,440","$15,605","$4,364","$28,576","$59,838","$27,411","$205,649","$45,610","$29,878","$2,602,247","$3,446,358"
16,2018-08,"$14,497","$25,815","$56,501","$47,342","$195,660","$11,273","$12,960","$4,246","$34,514","$34,446","$40,634","$222,949","$36,635","$19,931","$2,379,560","$3,136,964"
17,2018-09,$0,"$24,472","$28,073","$47,695","$243,299","$25,697","$13,792","$8,583","$4,820","$59,427","$67,362","$165,993","$27,972","$26,350","$2,331,235","$3,074,772"
18,2018-10,$0,"$27,997","$20,496","$59,440","$510,943",$0,"$21,536","$2,757","$53,722","$86,148","$112,710","$259,617","$24,346","$25,052","$2,956,808","$4,161,571"
19,2018-11,$0,"$67,504","$7,043","$70,133","$242,708",$0,"$19,492","$5,359","$29,712","$73,515","$47,064","$235,589","$42,992","$34,489","$2,996,261","$3,871,862"
20,2018-12,"$7,533","$24,491","$4,144","$55,759","$334,276",$0,"$20,223","$5,748","$11,021","$62,328","$42,701","$206,871","$39,510","$14,617","$2,289,051","$3,118,273"
21,2019-01,$0,"$32,644","$17,647","$59,317","$111,389",$0,"$18,798","$9,131","$37,095","$58,935","$91,893","$257,548","$27,377","$21,815","$2,659,867","$3,403,458"


### Compare totals with daily_stats

In [26]:
ds = pd.read_sql(
'''
SELECT
DATE_FORMAT(timestamp, '%Y-%m') AS year_and_month,
SUM(d_value) AS ds_revenue
FROM daily_stats
WHERE d_class = 'd_all'
AND DATE(timestamp) BETWEEN ' '''+ date_start +''' ' AND ' '''+ date_end +''' '
GROUP BY year_and_month
''', db)

col_fix(ds)

ds.set_index('year and month', inplace = True)

In [27]:
ot = pd.read_sql(
'''
SELECT
DATE_FORMAT(date_purchased, '%Y-%m') AS year_and_month,
SUM(value) AS ot_revenue
FROM orders_total ot
JOIN orders o ON ot.orders_id = o.orders_id
AND DATE(o.date_purchased) BETWEEN ' '''+ date_start +''' ' AND ' '''+ date_end +''' '
AND o.orders_status NOT IN (8,9,10,11,12,13,14,15)
AND o.payment_method != 'Replacement Order'
WHERE class = 'ot_subtotal'
GROUP BY year_and_month
''', db)

col_fix(ot)

ot.set_index('year and month', inplace = True)

### Check totals against other totals

In [28]:
totals = data_by_company.set_index('year and month')[['row total']].join(ot).join(ds)
totals.rename(columns = {'row total':'report revenue'}, inplace = True)

In [29]:
fmt = ['m0'] * 3
totals.tail().format_(fmt)

,Report Revenue,Ot Revenue,Ds Revenue
year and month,,,
2018-12,"$3,118,273","$3,120,026","$3,152,747"
2019-01,"$3,403,458","$3,403,928","$3,458,196"
2019-02,"$3,349,750","$3,349,937","$3,354,392"
2019-03,"$4,704,452","$4,705,145","$4,765,824"
2019-04,"$3,360,848","$3,361,197","$3,386,440"


In [30]:
(totals['report revenue'] - totals['ot revenue']).tail()

year and month
2018-12   -1752.840000
2019-01    -470.740000
2019-02    -186.670000
2019-03    -692.229997
2019-04    -349.420000
dtype: float64

### Compare structured vs unstuctured data values

In [31]:
val1 = orders_products.groupby('year and month')['revenue'].sum().values
val2 = data_by_company['row total'].values

check = pd.DataFrame(val1 - val2)
check[0] = [np.abs(x) for x in check[0]]

if check.loc[(check > 0.1).all(1)].empty:
    pass
else:
    raise 1

### Check Reseller totals

In [32]:
c1 = orders_products[orders_products['customers id'].isin(resellers.values())].groupby('customers id').agg({'orders id':'nunique','qty total':'sum'})
c1.rename(columns = {'orders id':'op order count',
                    'qty total':'op qty total'}, inplace = True)

c2 = data[data['customers id'].isin(resellers.values())].groupby('customers id').agg({'orders id':'nunique','qty total':'sum'})
c2.rename(columns = {'orders id':'data order count',
                    'qty total':'data qty total'}, inplace = True)

c3 = c1.div(c2.values, axis = 0)

# if you divide one by the other and the result for any element is NOT 1, then you missed something
if c3.loc[(c3 != 1).any(1)].empty:
    pass
else:
    raise 1

### The Five Tables and Excluded

In [33]:
do_not_stock = pd.read_excel(csv_path + 'Reseller Exclude.xlsx')

five_tables_dict = {}
excluded_dict = {}
unique_dict = {}

for k,v in resellers.items():
    
    name = k.split(' ')[0]
    dns_ls = do_not_stock[do_not_stock['reseller'] == k]['part id'].tolist()
    
    main = data[(~data['part id'].isin(always_exclude))
              & (~data['part id'].isin(dns_ls))
              & (data['date'] >= rolling_quarter_start)].copy()

    cols = ['part id',
           'products name',
           'bom',
           'combined sales 90',
           'link',
           'cost per unit',
           'msrp',
           'reseller discount qty',
           'reseller max % off',
           'qty discount qty',
           'qty max % off',
           'price per unit with reseller max discount',
           'margin with reseller max discount',
           'price per unit with qty max discount',
           'margin with qty max discount',
           'adafruit manufactured',
           'all customers',
           'reseller']
    
    df1 = main[cols].drop_duplicates()
    
    df2 = main.groupby('part id')[['qty bought']].sum()
    df1['qty bought'] = df1['part id'].map(dict(zip(df2.index.to_series(), df2['qty bought'])))
    
    
    ls = main[(main['date'] >= one_year_ago) & (main['customers id'] == v)]['part id'].tolist()
    df1['bought by ' + name + ' in the last year'] = np.where(df1['part id'].isin(ls),'x','')

    ls = main[(main['date'] >= rolling_quarter_start) & (main['customers id'] == v)]['part id'].tolist()
    df1['bought by ' + name + ' in the last three months'] = np.where(df1['part id'].isin(ls),'x','')

    ls = pd.read_sql(
    '''
    SELECT
    DISTINCT part_id
    FROM orders_products op
    JOIN orders o ON op.orders_id = o.orders_id
    AND customers_id = '''+ str(v) +'''
    ''', db)
    col_fix(ls)
    
    df1['never bought by ' + name] = np.where(~df1['part id'].isin(ls['part id'].tolist()),'x','')    
    
    if df1[(df1['bought by '+ name +' in the last year'] == 'x')
         & (df1['bought by '+ name +' in the last three months'] == 'x')
         & (df1['never bought by ' + name] == 'x')].empty:
        pass
    else:
        raise 1    
    
    df1['part id '] = df1['part id']
    df1['products name '] = df1['products name']
    df1.sort_values('part id', inplace = True)
    
    cols = df1.columns.tolist()
    cols.remove('qty bought')
    new_cols = cols[:2] + ['qty bought'] + cols[2:]
    df1 = df1[new_cols]    
    
    five_tables_dict[k] = df1
    
    ex = pd.read_sql(
    '''
    SELECT
    part_id,
    products_name
    FROM products_description
    WHERE part_id IN '''+ str(tuple(always_exclude + dns_ls)) +'''
    ''', db)

    col_fix(ex)
    
    ex['why'] = np.where(ex['part id'].isin(always_exclude), 'always exclude','do not stock')
    ex.sort_values('part id', inplace = True)
    excluded_dict[k] = ex
    
    un = data[(data['customers id'] == v) & (data['date'] >= one_year_ago)].copy()
    un.sort_values(['part id','date'], inplace = True)
    un.drop_duplicates('part id', keep = 'first', inplace = True)

    un2 = un.groupby('year and month')[['part id']].count()
    un3 = year_and_month.set_index('year and month').join(un2).fillna(0)

    un3['running sum'] = un3['part id'].cumsum()
    un3['m2m'] = un3['running sum'].pct_change()
    un3['yoy'] = un3['running sum'].pct_change(periods = 12)
    un3.drop('part id',1,inplace = True)
    un3.reset_index(inplace = True)    
    
    unique_dict[k] = un3

### Arrow example

In [34]:
five_tables_dict['Arrow'].head()

,part id,products name,qty bought,bom,combined sales 90,link,cost per unit,msrp,reseller discount qty,reseller max % off,qty discount qty,qty max % off,price per unit with reseller max discount,margin with reseller max discount,price per unit with qty max discount,margin with qty max discount,adafruit manufactured,all customers,reseller,bought by Arrow in the last year,bought by Arrow in the last three months,never bought by Arrow,part id,products name
1966093,9,High-strength 'rare earth' magnet,535,Resale - No Labor,4.3919,https://www.adafruit.com/product/9,0.19,2.50,25,0.3,100,0.2,1.750,0.891429,2.00,0.905000,,x,x,,,x,9,High-strength 'rare earth' magnet
2039907,14,MintyBoost Kit,883,kitted,8.4556,https://www.adafruit.com/product/14,1.08,19.50,50,0.3,100,0.2,13.650,0.920879,15.60,0.930769,,x,x,,,,14,MintyBoost Kit
1967805,16,Altoids Gum sized tin,1102,Resale - No Labor,10.8111,https://www.adafruit.com/product/16,0.65,2.00,50,0.3,100,0.2,1.400,0.535714,1.60,0.593750,,x,x,,,,16,Altoids Gum sized tin
1969712,18,USB/Serial Converter,526,Resale - No Labor,5.0889,https://www.adafruit.com/product/18,6.70,14.95,50,0.3,100,0.2,10.465,0.359771,11.96,0.439799,,x,x,,,,18,USB/Serial Converter
1969124,46,USBtinyISP AVR Programmer Kit (USB SpokePOV Dongle),109,kitted,0.8889,https://www.adafruit.com/product/46,5.22,22.00,50,0.3,100,0.2,15.400,0.661039,17.60,0.703409,,x,x,,,,46,USBtinyISP AVR Programmer Kit (USB SpokePOV Dongle)


In [35]:
excluded_dict['Arrow'].head()

,part id,products name,why
569,1,MiniPOV 2 Kit,always exclude
570,2,MiniPOV 2 PCB,always exclude
840,5,SpokePOV Kit,always exclude
845,6,SpokePOV Kit PCB,always exclude
850,7,SpokePOV Serial Dongle Kit,always exclude


In [36]:
unique_dict['Arrow']

,year and month,running sum,m2m,yoy
0,2018-04,50,NaN,NaN
1,2018-05,187,2.740000,NaN
2,2018-06,238,0.272727,NaN
3,2018-07,263,0.105042,NaN
4,2018-08,286,0.087452,NaN
5,2018-09,352,0.230769,NaN
6,2018-10,399,0.133523,NaN
7,2018-11,419,0.050125,NaN
8,2018-12,426,0.016706,NaN
9,2019-01,457,0.072770,NaN


### Excel start

In [37]:
if close_workbook == 'yes':
    
    workbook = xlsxwriter.Workbook(csv_path + workbook_title + '.xlsx',
                                   {'nan_inf_to_errors': False})
    
    # formats
    worksheet_title_format = workbook.add_format({'font_size':25,
                                                  'font_name':'Arial (Bold)'})

    header_and_column_format = workbook.add_format({'font_name':'Arial (Bold)',
                                                    'valign':'vcenter',
                                                    'align':'center',
                                                    'bg_color':'#edf9ff',
                                                    'bottom':1,
                                                    'top':1,
                                                    'left':1,
                                                    'right':1})

    money_format = workbook.add_format({'num_format':'$#,##0'})
    money_decimal_format = workbook.add_format({'num_format':'$#,##0.00'})

    percent_format = workbook.add_format({'num_format':'0.00%'})
    date_format = workbook.add_format({'num_format':'mmm yyyy'})

    number_format = workbook.add_format({'num_format':'#,##0'})


    # create reseller name list
    worksheet_list = ['Adafruit Stats']
    for key in resellers.keys():
        worksheet_list.append(key)

    # create worksheets
    for name in worksheet_list:
        workbook.add_worksheet(name)

    # send all to worksheet dict
    my_worksheets = {}
    for worksheet in workbook.worksheets():
        my_worksheets[worksheet.get_name()] = worksheet

    # do all similar formatting
    for key, value in my_worksheets.items():
        worksheet = my_worksheets[key]
        if key == 'Adafruit Stats':
            write_this = key + ' from ' + pretty_rolling_quarter_start + ' to ' + pretty_rolling_quarter_end
        else:
            write_this = key + ' with Customer ID: ' + str(resellers[key])
        worksheet.write(0, 0, 
                        write_this,
                        worksheet_title_format)

#===== adafruit data
#==============================================================================================================    

    worksheet = my_worksheets['Adafruit Stats']

    worksheet.merge_range(3, 1,
                          3, 3,
                         'All Customers',
                          header_and_column_format)

#===== all customers

    start_row = 4
    start_col = 1
    
    for col in range(len(all_cust.columns)):
        worksheet.write(start_row, start_col + col,
                       all_cust.columns[col].title(),
                       header_and_column_format)
        worksheet.set_column(start_col + col, start_col + col,
                         np.max([len(str(x)) for x in all_cust.iloc[:,col]] + [len(all_cust.columns[col])]) + 1)
        for row in range(len(all_cust)):
            if all_cust.columns[col] == 'qty bought':
                worksheet.write(start_row + 1 + row, start_col + col,
                               all_cust.iloc[row, col],
                               number_format)
            else:
                worksheet.write(start_row + 1 + row, start_col + col,
                               all_cust.iloc[row, col])

#===== resellers only

    worksheet.merge_range(3, 5,
                          3, 7,
                         'Resellers Only',
                          header_and_column_format)

    start_row = 4
    start_col = 5

    for col in range(len(res_only.columns)):
        worksheet.write(start_row, start_col + col,
                       res_only.columns[col].title(),
                       header_and_column_format)
        worksheet.set_column(start_col + col, start_col + col,
                         np.max([len(str(x)) for x in res_only.iloc[:,col]] + [len(res_only.columns[col])]) + 1)
        for row in range(len(res_only)):
            if res_only.columns[col] == 'qty bought':
                worksheet.write(start_row + 1 + row, start_col + col,
                               res_only.iloc[row, col],
                               number_format)
            else:
                worksheet.write(start_row + 1 + row, start_col + col,
                               res_only.iloc[row, col])

#===== adafruit manufactured only

    worksheet.merge_range(3, 9,
                          3, 11,
                         'Adafruit Manufactured Parts Only',
                          header_and_column_format)

    start_row = 4
    start_col = 9

    for col in range(len(ada_manufact_only.columns)):
        worksheet.write(start_row, start_col + col,
                       ada_manufact_only.columns[col].title(),
                       header_and_column_format)
        worksheet.set_column(start_col + col, start_col + col,
                         np.max([len(str(x)) for x in ada_manufact_only.iloc[:,col]] + [len(ada_manufact_only.columns[col])]) + 1)
        for row in range(len(ada_manufact_only)):
            if ada_manufact_only.columns[col] == 'qty bought':
                worksheet.write(start_row + 1 + row, start_col + col,
                               ada_manufact_only.iloc[row, col],
                               number_format)
            else:
                worksheet.write(start_row + 1 + row, start_col + col,
                               ada_manufact_only.iloc[row, col])

#===== data by company

    worksheet.merge_range(3, 13,
                          3, 13 + len(data_by_company.columns) - 1,
                         'Revenue by Reseller',
                          header_and_column_format)

    start_row = 4
    start_col = 13

    for col in range(len(data_by_company.columns)):
        worksheet.write(start_row, start_col + col,
                       data_by_company.columns[col].title(),
                       header_and_column_format)
        worksheet.set_column(start_col + col, start_col + col,
                         np.max([len(str(x)) for x in data_by_company.iloc[:,col]] + [len(data_by_company.columns[col])]) + 1)
        for row in range(len(data_by_company)):
            worksheet.write(start_row + 1 + row, start_col + col,
                           data_by_company.iloc[row, col],
                           money_format) 
            
#===== reseller data
#==============================================================================================================    

    for key, value in resellers.items():

        worksheet = my_worksheets[key]

        res_data = data.copy()
        res_data = res_data[res_data['customers id'] == value]

#===== revenue, gross profit, and qty sold
#===================================================================================================================

        numbers = res_data.groupby('year and month', as_index = False)[['revenue','gross profit','qty bought']].sum()
        numbers = pd.merge(year_and_month,
                          numbers,
                          how = 'left',
                          on = 'year and month')
        numbers.fillna(0, inplace = True)

        worksheet.merge_range(2, 1,
                              2, len(numbers.columns), 
                              'Revenue, Gross Profit, and Qty',
                              header_and_column_format)    
        start_row = 3
        start_col = 1

        for col in range(len(numbers.columns)):
            worksheet.write(start_row, start_col + col, 
                            numbers.columns[col].title(),
                            header_and_column_format)
            worksheet.set_column(start_col + col, start_col + col,
                             np.max([len(str(x)) for x in numbers.iloc[:,col]] + [len(numbers.columns[col])]) + 1)

            for row in range(len(numbers)):
                if numbers.columns[col] == 'year and month':
                    worksheet.write(start_row + 1 + row, start_col + col,
                                    numbers.iloc[row, col])
                elif numbers.columns[col] in ('revenue','gross profit'):
                    worksheet.write(start_row + 1 + row, start_col + col,
                                    numbers.iloc[row, col],
                                    money_format)
                elif numbers.columns[col] == 'qty bought':
                    worksheet.write(start_row + 1 + row, start_col + col,
                                    numbers.iloc[row, col],
                                    number_format)

#===== month to month and YoY % changes
#===================================================================================================================

        changes = numbers.set_index('year and month')
        changes = changes.pct_change()
        changes.loc['YoY'] = numbers.iloc[:,1:].pct_change(periods = 12).iloc[-1]
        changes.fillna(0,inplace = True)
        changes.replace([-np.inf,np.inf], 0, inplace = True)
        changes.reset_index(inplace = True)

        worksheet.merge_range(2, 6,
                              2, len(changes.columns) + 6 - 1, 
                              'Percent Changes',
                              header_and_column_format)    
        start_row = 3
        start_col = 6

        for col in range(len(changes.columns)):
            worksheet.write(start_row, start_col + col, 
                            changes.columns[col].title(),
                            header_and_column_format)
            worksheet.set_column(start_col + col, start_col + col,
                             np.max([len(str(x)) for x in changes.iloc[:,col]] + [len(changes.columns[col])]))

            for row in range(len(changes)):
                try:
                    worksheet.write(start_row + 1 + row, start_col + col,
                                   changes.iloc[row, col],
                                   percent_format)
                except:
                    worksheet.write_blank(start_row + 1 +  row, start_col + col,
                                          changes.iloc[row, col])

#===== descriptive stats by month
#==============================================================================================================                    

        describe = numbers.set_index('year and month').describe().reset_index().rename(columns = {'index':'statistic'}).copy()

        worksheet.merge_range(2, 11,
                              2, len(describe.columns) + 11 - 1, 
                              'Descriptive Statistics by Month',
                              header_and_column_format)    
        start_row = 3
        start_col = 11

        for col in range(len(describe.columns)):
            worksheet.write(start_row, start_col + col, 
                            describe.columns[col].title(),
                            header_and_column_format)
            worksheet.set_column(start_col + col, start_col + col,
                             np.max([len(str(x)) for x in describe.iloc[:,col]] + [len(describe.columns[col])]))

            for row in range(len(describe)):
                worksheet.write(start_row + 1 + row, start_col + col,
                               describe.iloc[row, col],
                               number_format)

#===== rolling quarter qty sold
#==============================================================================================================    

        qty = res_data[res_data['date'].between(rolling_quarter_start, rolling_quarter_end)].groupby(['part id','products name'], as_index = False)[['qty bought']].sum().sort_values('qty bought', ascending = False).head(50)    

        worksheet.merge_range(2, 16,
                              2, 18,
                             'top ' + str(len(qty)) + ' parts in last 3 months' if len(qty) < 50 else 'top 50 parts bought in last 3 months',
                             header_and_column_format)
        start_row = 3
        start_col = 16

        for col in range(len(qty.columns)):
            worksheet.write(start_row, start_col + col,
                           qty.columns[col].title(),
                           header_and_column_format)
            worksheet.set_column(start_col + col, start_col + col,
                             np.max([len(str(x)) for x in qty.iloc[:,col]] + [len(qty.columns[col])]))        
            for row in range(len(qty)):
                if qty.columns[col] == 'qty bought':
                    worksheet.write(start_row + 1 + row, start_col + col,
                                   qty.iloc[row, col],
                                   number_format)
                else:
                    worksheet.write(start_row + 1 + row, start_col + col,
                                   qty.iloc[row, col])            
                    
#===== the five tables
#==============================================================================================================                    

        five_tables = five_tables_dict[key].copy()

        worksheet.merge_range(2, 20,
                              2, 20 + len(five_tables.columns) - 1,
                             'The Five Tables',
                             header_and_column_format)

        start_row = 3
        start_col = 20

        for col in range(len(five_tables.columns)):
            worksheet.write(start_row, start_col + col,
                           five_tables.columns[col].title(),
                           header_and_column_format)
            worksheet.set_column(start_col + col, start_col + col,
                     np.max([len(str(x)) for x in five_tables.iloc[:,col]] + [len(five_tables.columns[col])]) + 10)

            for row in range(len(five_tables)):
                try:
                    if five_tables.columns[col] in ('msrp',
                                                    'cost per unit',
                                                    'lowest price @ 30% off',
                                                    'price per unit with reseller max discount',
                                                    'price per unit with qty max discount'):
                        worksheet.write(start_row + 1 + row, start_col + col,
                                       five_tables.iloc[row, col],
                                       money_decimal_format)
                    elif five_tables.columns[col] in ('reseller max % off',
                                                      'margin with reseller max discount',
                                                      'qty max % off',
                                                      'margin with qty max discount'):
                        worksheet.write(start_row + 1 + row, start_col + col,
                                       five_tables.iloc[row, col],
                                       percent_format)        
                    elif five_tables.columns[col] in ('part id',
                                                      'qty bought',
                                                      'reseller discount qty',
                                                      'qty discount qty',
                                                      'combined sales 90'):
                        worksheet.write(start_row + 1 + row, start_col + col,
                                       five_tables.iloc[row, col],
                                       number_format)                        
                    else:
                        worksheet.write(start_row + 1 + row, start_col + col,
                                       five_tables.iloc[row, col])
                except:
                    worksheet.write_blank(start_row + 1 + row, start_col + col,
                                   five_tables.iloc[row, col])


        worksheet.autofilter(start_row, start_col,
                            start_row + len(five_tables), start_col + len(five_tables.columns) - 1)                    

#===== excluded
#==============================================================================================================                    

        excluded = excluded_dict[key].copy()

        worksheet.merge_range(2, 45,
                              2, 47,
                             'Excluded Parts',
                             header_and_column_format)

        start_row = 3
        start_col = 45

        for col in range(len(excluded.columns)):
            worksheet.write(start_row, start_col + col,
                           excluded.columns[col].title(),
                           header_and_column_format)
            worksheet.set_column(start_col + col, start_col + col,
                     np.max([len(str(x)) for x in excluded.iloc[:,col]] + [len(excluded.columns[col])]) + 10)
            for row in range(len(excluded)):
                worksheet.write(start_row + 1 + row, start_col + col,
                               excluded.iloc[row, col])
        
#===== unique part count
#==============================================================================================================                    
        
        unique_parts = unique_dict[key].copy()

        worksheet.merge_range(2, 49,
                              2, 52,
                             'Unique Part Count',
                             header_and_column_format)

        start_row = 3
        start_col = 49

        for col in range(len(unique_parts.columns)):
            worksheet.write(start_row, start_col + col,
                           unique_parts.columns[col].title(),
                           header_and_column_format)
            worksheet.set_column(start_col + col, start_col + col,
                     np.max([len(str(x)) for x in unique_parts.iloc[:,col]] + [len(unique_parts.columns[col])]) + 5)
            for row in range(len(unique_parts)):
                try:
                    if unique_parts.columns[col] == 'running sum':
                        worksheet.write(start_row + 1 + row, start_col + col,
                                       unique_parts.iloc[row, col],
                                       number_format)
                    elif unique_parts.columns[col] in ('m2m','yoy'):
                        worksheet.write(start_row + 1 + row, start_col + col,
                                       unique_parts.iloc[row, col],
                                       percent_format)
                    else:
                        worksheet.write(start_row + 1 + row, start_col + col,
                                       unique_parts.iloc[row, col])                    
                except:
                        worksheet.write_blank(start_row + 1 + row, start_col + col,
                                       unique_parts.iloc[row, col])        
        
        
    workbook.close()

/anaconda3/lib/python3.6/site-packages/xlsxwriter/worksheet.py:4675: RuntimeWarning: invalid value encountered in double_scalars
  return (x - x) != 0


In [38]:
import xlwings as xw

if close_plot == 'yes':

    sht = xw.Book(csv_path + workbook_title + '.xlsx').sheets

#===== reseller specific charts        
#===================================================================================================================
    
    colors = {
    'color01':'#085f89',
    'color02':'#038fd2',
    'color03':'#edf9ff',
    'color04':'#989898',
    'color05':'#313131'}

    for key, value in resellers.items():

        res_data = data.copy()
        res_data = res_data[res_data['customers id'] == value]

        numbers = res_data.groupby('year and month', as_index = False)[['revenue','gross profit','qty bought']].sum()
        numbers = pd.merge(year_and_month,
                          numbers,
                          how = 'left',
                          on = 'year and month')
        numbers.fillna(0, inplace = True)

        for_reseller_chart = numbers.copy()
        for_reseller_chart.set_index('year and month', inplace = True)
        for_reseller_chart.columns = [x.title() for x in for_reseller_chart.columns]
        
        reseller_chart, ax = plt.subplots(figsize = (15,5))

        for_reseller_chart[['Revenue','Gross Profit']].plot(ax = ax,
                                                           color = [colors['color01'],colors['color04']],
                                                           kind = 'bar',
                                                           edgecolor = 'black',
                                                           lw = 2,
                                                           width = 0.75)

        ax.set_facecolor(colors['color03'])
        ax.set_xticks(np.arange(len(for_reseller_chart)))
        ax.set_xticklabels([calendar.month_abbr[int(str(x)[-2:])] + '\n' + str(x)[:4] for x in for_reseller_chart.index],
                          rotation = 0,
                          fontweight = 'bold',
                          fontsize = 15)
        ax.set_xlabel('')
        vals = ax.get_yticks()
        ax.set_yticklabels(['${:,.0f}'.format(x/1000) for x in vals], fontweight = 'bold')
        ax.set_ylabel('Thousands', fontsize = 15, fontweight = 'bold')
        ax.set_title(key + ' Revenue and Gross Profit',
                    y = 1.02,
                    fontsize = 25,
                    fontweight = 'bold')

        x = range(len(for_reseller_chart))
        y1 = for_reseller_chart['Revenue']
        y2 = for_reseller_chart['Gross Profit']

        z1 = np.polyfit(x, y1, 1)
        p1 = np.poly1d(z1)

        z2 = np.polyfit(x, y2, 1)
        p2 = np.poly1d(z2)

        ax.plot(x, p1(x), color = colors['color01'], ls = '--', lw = 3, label = 'Revenue Trend')
        ax.plot(x, p2(x), color = colors['color04'], ls = '--', lw = 3, label = 'Gross Profit Trend')    

        ax.grid()
        ax.legend(edgecolor = 'black')
            
        sht_plot = sht[key].pictures.add(reseller_chart, 
                                    name = 'reseller_chart', 
                                    update = True, 
                                    left = sht[key].range('B21').left, 
                                    top = sht[key].range('B21').top)

        plt.close(reseller_chart)    
        
#===== all resellers chart for first worksheet
#===================================================================================================================

        all_resellers = data_by_company.set_index('year and month').copy()
        all_resellers.drop(['the rest','row total'],1,inplace = True)
        all_resellers['total'] = all_resellers.sum(1)
        all_resellers['mean'] = all_resellers.mean(1)
        all_resellers = all_resellers[['total','mean']]
        all_resellers.columns = [x.title() for x in all_resellers]

        all_resellers_chart, ax = plt.subplots(figsize = (16,5))

        all_resellers['Total'].plot(kind = 'bar',
                               ax = ax,
                               color = colors['color01'],
                               edgecolor = 'black',
                               lw = 2,
                               zorder = 5)
        all_resellers['Mean'].plot(ax = ax, 
                              color = colors['color02'],
                              ls = '--',
                              lw = 3,
                              marker = 'o',
                              mew = 4,
                              zorder = 10)

        ax.set_facecolor(colors['color03'])
        ax.set_xticks(np.arange(len(all_resellers)))
        ax.set_xticklabels([calendar.month_abbr[int(str(x)[-2:])] + '\n' + str(x)[:4] for x in all_resellers.index],
                          rotation = 0,
                          fontsize = 10,
                          fontweight = 'bold')
        ax.set_xlabel('')
        vals = ax.get_yticks()
        ax.set_yticklabels(['${:,.0f}'.format(x/1000) for x in vals], fontweight = 'bold')
        ax.set_ylabel('Thousands', fontsize = 15, fontweight = 'bold')
        ax.set_title('Total and Avg Revenue from Resellers', fontsize = 25, fontweight = 'bold', y = 1.02)
        ax.grid()
        ax.legend(edgecolor = 'black', fontsize = 20)

        sht_plot = sht['Adafruit Stats'].pictures.add(all_resellers_chart, 
                                    name = 'all_resellers_chart', 
                                    update = True, 
                                    left = sht['Adafruit Stats'].range('Q33').left, 
                                    top = sht['Adafruit Stats'].range('Q33').top)

        plt.close(all_resellers_chart)        

In [39]:
print('done')

done
